In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
# evaluate the model 
from sklearn.metrics import classification_report



In [17]:
df = pd.read_csv('classification_cleaned.csv')
df.head()

,alpha,delta,cam_col,class,redshift,plate,MJD,photometric_filters
0,135.689107,32.494632,2,0,0.634794,5812,56354,21.422417
1,144.826101,31.274185,5,0,0.779136,10445,58158,22.918595
2,142.188790,35.582444,2,0,0.644195,4576,55592,21.999873
3,338.741038,-0.402828,3,0,0.932346,9149,58039,21.768664
4,345.282593,21.183866,3,0,0.116123,6121,56187,17.380674


In [48]:
df.shape

(100000, 8)

In [49]:
# create X and y
X = df.drop('class', axis=1)
y = df['class']

In [50]:
df['class'].value_counts()

class
0    59445
2    21594
1    18961
Name: count, dtype: int64

In [51]:
# Using the under-sampling method to balance the values of the class column as you can see it previously. all those three values are not balanced
rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(X, y)


In [83]:
# after using the under-sampling method, lt's check the value counts of the 'class' column
pd.Series(y).value_counts()


class
0    18961
1    18961
2    18961
Name: count, dtype: int64

In [82]:
print("X_train shape:", X.shape)
print("y_train shape:", y.shape)

X_train shape: (56883, 7)
y_train shape: (56883,)


In [90]:
# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [91]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(39818, 7)
(39818,)
(17065, 7)
(17065,)


In [92]:
# Creating of the DMatrix object 
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)  

In [93]:
# creating the parameters 
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',
    'num_class': 3,
    'tree_method': 'hist',
}


In [94]:
bst = xgb.train(params, dtrain)


In [95]:
preds = bst.predict(dtest)


In [96]:
# Ensure preds contains predictions for all samples in y_test
preds = bst.predict(dtest)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5725
           1       0.97      0.88      0.92      5669
           2       0.99      1.00      0.99      5671

    accuracy                           0.95     17065
   macro avg       0.95      0.95      0.95     17065
weighted avg       0.95      0.95      0.95     17065

